1) Показать ТОП-5 самых продаваемых гербицидов/фунгицидов/инсектицидов по площади обработки в 2015-2020 гг. с разбивкой по годам (*найти их состав).

2) Показать среднюю стоимость обработки гербицидами, фунгицидами и инсектицидами 1 га зерновых в 2015-2020 гг. с разбивкой по годам.

3) Показать ТОП-5 самых дорогих гербицидов/фунгицидов/инсектицидов в 2015-2020 гг. с разбивкой по годам (*найти текущие цены на данные продукты).

4) Показать динамику применения гербицидов/фунгицидов/инсектицидов в 2015-2020 гг. по площади и по количеству.

5) По ТОП-5 самых продаваемых гербицидов/фунгицидов/инсектицидов в 2020 году найти, есть ли корреляция между ценой и продаваемым количеством.

In [7]:
import pandas as pd;

In [3]:
File_path = 'C:/Users/User/Downloads/Пестициды на зерновые_тестовое задание.xlsx'

# Финализируем решение

In [4]:
type_list = ['herbicide','Fungicide','Insecticides']

In [5]:
for ii in range(len(type_list)):
    typ = type_list[ii]
    
    
    xl = pd.ExcelFile(File_path)
    xll = xl.sheet_names 
    data = pd.read_excel(File_path,sheet_name=xll[ii],skiprows=[0,1])

    data.head()
    df1= pd.DataFrame(data['Unnamed: 0'][1:])

    dataclass = data.iloc[0].dropna().unique()

    k = len(data) - 1

    d = {} #Создаем словарь в котором будем хранить датафреймы, с разбивкой dataclass
    for i in range(len(dataclass)):

        x = dataclass[i]
        datat = data.iloc[:, i*6+1:i*6+7][1:]
        d["{0}".format(x)] = datat.rename(columns=datat.iloc[0]).fillna(0)
        # Заменим пропущенные значения на 0, так как потом это все равно придётся делать 

    S = df1.join(d['Площадь обработки, тыс. га']).fillna(0)

    dST = {} #Создаем словарь в котором будем хранить датафреймы, с разбивкой dataclass
    for i in range(len(dataclass)):

        x = dataclass[i]
        datat = data.iloc[:, i*6+1:i*6+7][1:]
        d["{0}".format(x)] = datat.rename(columns=datat.iloc[0]).fillna(0)
        # Заменим пропущенные значения на 0, так как потом это все равно придётся делать 


    for i in range(6):
        x = 2015 + i
        SalesTop = S.iloc[:, [0, i+1]][2:-1].set_axis(['Brand', 'Value'], axis=1, inplace=False).sort_values(by=['Value'],ascending=False)[:5].reset_index(drop=True)
        dST["{0}".format(x)] = SalesTop.rename(columns=datat.iloc[0]).fillna(0)


    SAgg = d[dataclass[1]].loc[[k][:]].set_axis(['2015', '2016', '2017', '2018', '2019', '2020'], axis=1, inplace=False) 
    PAgg = d[dataclass[2]].loc[[k][:]].set_axis(['2015', '2016', '2017', '2018', '2019', '2020'], axis=1, inplace=False) 

    AP = PAgg/SAgg
    AP

    #Динамика средних цен в миллионах рублей на обработку одного гектара

    SAgg = d[dataclass[1]].loc[2:k,:].set_axis(['2015', '2016', '2017', '2018', '2019', '2020'], axis=1, inplace=False).reset_index(drop=True).fillna(0) 
    PAgg = d[dataclass[2]].loc[2:k,:].set_axis(['2015', '2016', '2017', '2018', '2019', '2020'], axis=1, inplace=False).reset_index(drop=True).fillna(0)

    #Динамика средних цен в миллионах рублей на обработку одного гектара
    ans= PAgg/SAgg

    ansn = df1.loc[2:k,:].join(ans).fillna(0)


    dE = {} #Создаем словарь в котором будем хранить датафреймы, с разбивкой dataclass


    for i in range(6):
        x = 2015 + i
        Expensive = ansn.iloc[:, [0, i+1]][2:-1].set_axis(['Brand', 'Value'], axis=1, inplace=False).sort_values(by=['Value'],ascending=False)[:5].reset_index(drop=True)
        dE["{0}".format(x)] = SalesTop.rename(columns=datat.iloc[0]).fillna(0)


    dynamicsValue = d[dataclass[0]].loc[[k][:]].set_axis(['2015', '2016', '2017', '2018', '2019', '2020'], axis=1, inplace=False).reset_index(drop=True) 
    #Динамика по колличеству купленных, в тоннах
    dynamicsArea =  d[dataclass[1]].loc[[k][:]].set_axis(['2015', '2016', '2017', '2018', '2019', '2020'], axis=1, inplace=False).reset_index(drop=True) 
    #Динамика по площади обработки, в тысячах гектаров



    S2020 = S.iloc[:, [0, 5]][2:-1].set_axis(['Brand', 'Value'], axis=1, inplace=False).sort_values(by=['Value'],ascending=False)[:5]
    Profit = df1.join(d['Выручка, млн. руб.']).fillna(0)
    Profit2020 = Profit.iloc[:, [0, 6]]
    ProfitValue = S2020.join(Profit2020).iloc[:, [0, 1,3]].set_axis(['Brand', 'Value','Profit'], axis=1, inplace=False)


    PVC = ProfitValue.corr()
    # Корелляция пирсона объема и дохода ~0.78 - значит она есть
    
    
    
    
    with pd.ExcelWriter(f'C:/Users/User/Downloads/Lysterra_Task{typ}.xlsx') as writer: # Переводим в Excel, что бы можно было удобно смотреть менеджерам

            
            for df_name, df in dST.items():
                df.to_excel(writer, sheet_name= f'{typ}_{df_name}_ST') # 1

            AP.to_excel(writer, sheet_name= f'{typ}_AvPrice') # 2

            for df_name, df in dE.items():
                df.to_excel(writer, sheet_name= f'{typ}_{df_name}_MostExpensive')#3

            dynamicsValue.to_excel(writer, sheet_name= f'{typ}_DynamicsValue')#4
            dynamicsArea.to_excel(writer, sheet_name= f'{typ}_DynamicsArea')#4

            PVC.to_excel(writer, sheet_name= f'{typ}_Correlation')#5

In [6]:
# Тоже самое можно сделать и в одной ячейке 